# SEIR-HCD Model
This is a working example of a [SIER](https://en.wikipedia.org/wiki/Compartmental_models_in_epidemiology#The_SEIR_model) model with added compartments for HCD. The letters stand for:
* Susceptible
* Exposed
* Infected
* Recovered
* Hospitalized
* Critical
* Death

I have adapted the equations from these great web apps:
* http://gabgoh.github.io/COVID/index.html
* https://neherlab.org/covid19

**NOTE:** If you are looking for the SIER model, check commit 20 or earlier.

In [ ]:
from IPython.display import Image
from IPython.core.display import HTML
Image(url= "https://pbs.twimg.com/media/EUJNQQDXsAwz0tK?format=jpg")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import os
from tqdm.notebook import tqdm
from scipy.integrate import solve_ivp
from scipy.optimize import minimize
from sklearn.metrics import mean_squared_log_error, mean_squared_error

## Parameters used in the model
`R_t` = reproduction number at time t. Typical 2.2 at t=0

**Transition times**
* `T_inc` = average incubation period. Typical 5.2 days
* `T_inf` = average infectious period. Typical 2.9 days
* `T_hosp` = average time a patient is in hospital before either recovering or becoming critical. Typical 4 days
* `T_crit` = average time a patient is in a critical state (either recover or die). Typical 14 days

**Fractions**
These constants are likely to be age specific (hence the subscript a):
* `m_a` = fraction of infections that are asymptomatic or mild. Assumed 80% (i.e. 20% severe)
* `c_a` = fraction of severe cases that turn critical. Assumed 10%
* `f_a` = fraction of critical cases that are fatal. Assumed 30%

*TODO*: Find sources for these numbers

In [ ]:
# Original
PARAM_BOUNDS_SAFE = ((1., 5.5), # R bounds
                (0.5, 10), (2, 20), # transition time param bounds
                (0.5, 1), (0.01, 0.30), (0.1, 0.6), (1, 5), (1, 100)) # fraction time param bounds

PARAM_BOUNDS = ((1, 20), # R bounds
                (0.5, 10), (2, 20), # transition time param bounds
                (0.5, 1), (0, 1), (0, 1), (1, 5), (1, 100)) # fraction time param bounds

OPTIM_DAYS = 14  # Number of days to use for the optimisation evaluation

In [ ]:
# Susceptible equation
def dS_dt(S, I, R_t, t_inf):
    return -(R_t / t_inf) * I * S


# Exposed equation
def dE_dt(S, E, I, R_t, t_inf, t_inc):
    return (R_t / t_inf) * I * S - (E / t_inc)


# Infected equation
def dI_dt(I, E, t_inc, t_inf):
    return (E / t_inc) - (I / t_inf)


# Hospialized equation
def dH_dt(I, C, H, t_inf, t_hosp, t_crit, m_a, f_a):
    return ((1 - m_a) * (I / t_inf)) + ((1 - f_a) * C / t_crit) - (H / t_hosp)


# Critical equation
def dC_dt(H, C, t_hosp, t_crit, c_a):
    return (c_a * H / t_hosp) - (C / t_crit)


# Recovered equation
def dR_dt(I, H, t_inf, t_hosp, m_a, c_a):
    return (m_a * I / t_inf) + (1 - c_a) * (H / t_hosp)


# Deaths equation
def dD_dt(C, t_crit, f_a):
    return f_a * C / t_crit


def SEIR_HCD_model(t, y, R_t, t_inc=2.9, t_inf=5.2, t_hosp=4, t_crit=14, m_a=0.8, c_a=0.1, f_a=0.3):
    """

    :param t: Time step for solve_ivp
    :param y: Previous solution or initial values
    :param R_t: Reproduction number
    :param t_inc: Average incubation period. Default 5.2 days
    :param t_inf: Average infectious period. Default 2.9 days
    :param t_hosp: Average time a patient is in hospital before either recovering or becoming critical. Default 4 days
    :param t_crit: Average time a patient is in a critical state (either recover or die). Default 14 days
    :param m_a: Fraction of infections that are asymptomatic or mild. Default 0.8
    :param c_a: Fraction of severe cases that turn critical. Default 0.1
    :param f_a: Fraction of critical cases that are fatal. Default 0.3
    :return:
    """
    if callable(R_t):
        reprod = R_t(t)
    else:
        reprod = R_t
        
    S, E, I, R, H, C, D = y
    
    S_out = dS_dt(S, I, reprod, t_inf)
    E_out = dE_dt(S, E, I, reprod, t_inf, t_inc)
    I_out = dI_dt(I, E, t_inc, t_inf)
    R_out = dR_dt(I, H, t_inf, t_hosp, m_a, c_a)
    H_out = dH_dt(I, C, H, t_inf, t_hosp, t_crit, m_a, f_a)
    C_out = dC_dt(H, C, t_hosp, t_crit, c_a)
    D_out = dD_dt(C, t_crit, f_a)
    return [S_out, E_out, I_out, R_out, H_out, C_out, D_out]

In [ ]:
# # Susceptible equation
# def dS_dt(S, I, R_t, t_inf):
#     return -(R_t / t_inf) * I * S


# # Exposed equation
# def dE_dt(S, E, I, R_t, t_inf, t_inc):
#     return (R_t / t_inf) * I * S - (E / t_inc)


# # Infected equation
# def dI_dt(I, E, t_inc, t_inf):
#     return (E / t_inc) - (I / t_inf)


# # Hospialized equation
# def dH_dt(I, C, H, t_inf, t_hosp, t_crit, m_a, f_a):
#     return ((1 - m_a) * (I / t_inf)) + ((1 - f_a) * C / t_crit) - (H / t_hosp)


# # Critical equation
# def dC_dt(H, C, t_hosp, t_crit, c_a):
#     return (c_a * H / t_hosp) - (C / t_crit)


# # Recovered equation
# def dR_dt(I, H, t_inf, t_hosp, m_a, c_a):
#     return (m_a * I / t_inf) + (1 - c_a) * (H / t_hosp)


# # Deaths equation
# def dD_dt(C, t_crit, f_a):
#     return f_a * C / t_crit


# def SEIR_HCD_model(t, y, R_t, t_inc=2.9, t_inf=5.2, t_hosp=4, t_crit=14, m_a=0.8, c_a=0.1, f_a=0.3):
#     """

#     :param t: Time step for solve_ivp
#     :param y: Previous solution or initial values
#     :param R_t: Reproduction number
#     :param t_inc: Average incubation period. Default 5.2 days
#     :param t_inf: Average infectious period. Default 2.9 days
#     :param t_hosp: Average time a patient is in hospital before either recovering or becoming critical. Default 4 days
#     :param t_crit: Average time a patient is in a critical state (either recover or die). Default 14 days
#     :param m_a: Fraction of infections that are asymptomatic or mild. Default 0.8
#     :param c_a: Fraction of severe cases that turn critical. Default 0.1
#     :param f_a: Fraction of critical cases that are fatal. Default 0.3
#     :return:
#     """
#     if callable(R_t):
#         reprod = R_t(t)
#     else:
#         reprod = R_t
        
#     S, E, I, R, H, C, D = y
    
#     S_out = dS_dt(S, I, reprod, t_inf)
#     E_out = dE_dt(S, E, I, reprod, t_inf, t_inc)
#     I_out = dI_dt(I, E, t_inc, t_inf)
#     R_out = dR_dt(I, H, t_inf, t_hosp, m_a, c_a)
#     H_out = dH_dt(I, C, H, t_inf, t_hosp, t_hosp + t_crit, m_a, f_a)
#     C_out = dC_dt(H, C, t_hosp, t_hosp + t_crit, c_a)
#     D_out = dD_dt(C, t_hosp + t_crit, f_a)
#     return [S_out, E_out, I_out, R_out, H_out, C_out, D_out]

In [ ]:
def plot_model(solution, title='SEIR+HCD model'):
    sus, exp, inf, rec, hosp, crit, death = solution.y
    
    cases = inf + rec + hosp + crit + death

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16,5))
    fig.suptitle(title)
    
    ax1.plot(sus, 'tab:blue', label='Susceptible');
    ax1.plot(exp, 'tab:orange', label='Exposed');
    ax1.plot(inf, 'tab:red', label='Infected');
    ax1.plot(rec, 'tab:green', label='Recovered');
    ax1.plot(hosp, 'tab:purple', label='Hospitalised');
    ax1.plot(crit, 'tab:brown', label='Critical');
    ax1.plot(death, 'tab:cyan', label='Deceased');
    
    ax1.set_xlabel("Days", fontsize=10);
    ax1.set_ylabel("Fraction of population", fontsize=10);
    ax1.legend(loc='best');
    
    ax2.plot(cases, 'tab:red', label='Cases');    
    ax2.set_xlabel("Days", fontsize=10);
    ax2.set_ylabel("Fraction of population (Cases)", fontsize=10, color='tab:red');
    
    ax3 = ax2.twinx()
    ax3.plot(death, 'tab:cyan', label='Deceased');    
    ax3.set_xlabel("Days", fontsize=10);
    ax3.set_ylabel("Fraction of population (Fatalities)", fontsize=10, color='tab:cyan');


# Model without intervention
Let's see what the model looks like without any intervention, i.e. `R_0` is a contant value

In [ ]:
N = 100000  # Population size
n_infected = 1
max_days = 200

# State at time = 0 for SEIR_HCD model
# The numbers correspond to the number of people in each of the SEIRHCD compartments
initial_state = [(N - n_infected)/ N, 0, n_infected / N, 0, 0, 0, 0]

R_0 = 2.2
t_inc = 5.2
t_inf = 2.9
t_hosp = 4
t_crit = 14
m_a = 0.8
c_a = 0.1
f_a = 0.3

args = (R_0, t_inc, t_inf, t_hosp, t_crit, m_a, c_a, f_a)

sol = solve_ivp(SEIR_HCD_model, [0, max_days], initial_state, args=args, t_eval=np.arange(max_days))

plot_model(sol, 'SEIR-HCD Model (without intervention)')

# Model with intervention
Lets assume that there is some intervention that causes the reproduction number (`R_0`) to fall to a lower value (`R_t`) at a certain time (e.g. physical distancing). Note that the actual drop will occur some time after the intervention measures are implemented.

This could be modified to take any function of `R_t(t)` values to model the reproduction number as a time varying variable

In [ ]:
R_0 = 2.2  # reproduction number without intervention
R_t = 0.7  # reproduction number after intervention
intervention_day = 80

def time_varying_reproduction(t):
    if t > intervention_day:
        return R_t
    else:
        return R_0
    
args = (time_varying_reproduction, t_inc, t_inf, t_hosp, t_crit, m_a, c_a, f_a)

sol2 = solve_ivp(SEIR_HCD_model, [0, max_days], initial_state, args=args, t_eval=np.arange(max_days))

plot_model(sol2, f'SEIR-HCD Model (with intervention on day {intervention_day})')

Let's compare the infection rate between the two cases

In [ ]:
_, exp, inf, _, _, _, deaths = sol.y
_, exp2, inf2, _, _, _, deaths2 = sol2.y

f = plt.figure(figsize=(8,5)) 
# plt.plot(exp, 'tab:orange', label='Exposed', linestyle=':');
# plt.plot(exp2, 'tab:orange', label='Exposed with intervention');
plt.plot(inf, 'r', label='Infected', linestyle=':');
plt.plot(inf2, 'r', label='Infected with intervention');
plt.plot(deaths, 'b', label='Deceased', linestyle=':');
plt.plot(deaths2, 'b', label='Deceased with intervention');

plt.title(f'Comparison of the effect of the intervention on day {intervention_day}')
plt.xlabel("Days", fontsize=10);
plt.ylabel("Fraction of population", fontsize=10);
plt.legend(loc='best');

You can see that after the intervention on day 80, the peak infections is lower than if there was no intervention and there are less than half as many deaths. You can see how powerful self-isolation is from this chart

# Fitting the model to data
There are certain variables that we can play with to fit the model to real data:
* Average incubation period, `t_inc`
* Average infection period, `t_inf`
* Average hospitalization period, `t_hosp`
* Average critital period, `t_crit`
* The fraction of mild/asymptomatic cases, `m_a`
* The fraction of severe cases that turn critical, `c_a`
* The fraction of critical cases that result in a fatality, `f_a`
* Reproduction number, `R_0` or `R_t`

The some of these are likely to be constants specific to the virus and some are likely to be time dependent variables dependent on factors such as:
* When a government intervened
* Peoples behaviours (do people actively self-isolate, not visit religious shrines etc.)
* Population demographic of a country (is a significant proportion of the population old?). This is the `a` subscript
* Heathcare system capacity (hostpital beds per capita)
* Number of testing kits available

We have already used two different reproduction numbers above. Let's see if we can derive a time-dependent `R_t` from the data. We will also try and optimize a handful of the parameters above to match the data.

We will also compare this to just using a single reproduction number. This might actaully be more suitable in countries where the outbreak has just started or they are struggling to limit the spread.

There are lots of ways to decay a parameter in epidemiology. I'm going to use a Hill decay, which has 2 parameters, `k` and `L` (the half decay constant):

In [ ]:
Image(url= "https://raw.githubusercontent.com/wiki/SwissTPH/openmalaria/img/graphs/decay-functions.png")

In [ ]:
DATE_BORDER = '2020-04-01'

data_path = Path('/kaggle/input/covid19-global-forecasting-week-2/')

train = pd.read_csv(data_path / 'train.csv', parse_dates=['Date'])
test = pd.read_csv(data_path /'test.csv', parse_dates=['Date'])
submission = pd.read_csv(data_path /'submission.csv', index_col=['ForecastId'])

# Load the population data into lookup dicts
pop_info = pd.read_csv('/kaggle/input/covid19-population-data/population_data.csv')
country_pop = pop_info.query('Type == "Country/Region"')
province_pop = pop_info.query('Type == "Province/State"')
country_lookup = dict(zip(country_pop['Name'], country_pop['Population']))
province_lookup = dict(zip(province_pop['Name'], province_pop['Population']))

# Fix the Georgia State/Country confusion - probably a better was of doing this :)
train['Province_State'] = train['Province_State'].replace('Georgia', 'Georgia (State)')
test['Province_State'] = test['Province_State'].replace('Georgia', 'Georgia (State)')
province_lookup['Georgia (State)'] = province_lookup['Georgia']

train['Area'] = train['Province_State'].fillna(train['Country_Region'])
test['Area'] = test['Province_State'].fillna(test['Country_Region'])

# https://www.kaggle.com/c/covid19-global-forecasting-week-1/discussion/139172
train['ConfirmedCases'] = train.groupby('Area')['ConfirmedCases'].cummax()
train['Fatalities'] = train.groupby('Area')['Fatalities'].cummax()

# Remove the leaking data
train_full = train.copy()
valid = train[train['Date'] >= test['Date'].min()]
train = train[train['Date'] < test['Date'].min()]

# Split the test into public & private
test_public = test[test['Date'] <= DATE_BORDER]
test_private = test[test['Date'] > DATE_BORDER]

# Use a multi-index for easier slicing
train_full.set_index(['Area', 'Date'], inplace=True)
train.set_index(['Area', 'Date'], inplace=True)
valid.set_index(['Area', 'Date'], inplace=True)
test_public.set_index(['Area', 'Date'], inplace=True)
test_private.set_index(['Area', 'Date'], inplace=True)

submission['ConfirmedCases'] = 0
submission['Fatalities'] = 0

train_full.shape, train.shape, valid.shape, test_public.shape, test_private.shape, submission.shape

In [ ]:
valid.index.get_level_values(1).max()

The function below evaluates a model with a constant `R` number as well as `t_hosp`, `t_crit`, `m`, `c`, `f`

In [ ]:
# Use a constant reproduction number
def eval_model_const(params, data, population, return_solution=False, forecast_days=0, optim_days=OPTIM_DAYS):
    R_0, t_hosp, t_crit, m, c, f = params
    N = population
    n_infected = data['ConfirmedCases'].iloc[0]
    max_days = len(data) + forecast_days
    initial_state = [(N - n_infected)/ N, 0, n_infected / N, 0, 0, 0, 0]
    args = (R_0, 5.2, 2.9, t_hosp, t_crit, m, c, f)
               
    sol = solve_ivp(SEIR_HCD_model, [0, max_days], initial_state, args=args, t_eval=np.arange(0, max_days))
    
    sus, exp, inf, rec, hosp, crit, deaths = sol.y
    
    y_pred_cases = np.clip(inf + rec + hosp + crit + deaths, 0, np.inf) * population
    y_true_cases = data['ConfirmedCases'].values
    y_pred_fat = np.clip(deaths, 0, np.inf) * population
    y_true_fat = data['Fatalities'].values
    
    optim_days = min(optim_days, len(data))  # Days to optimise for
    weights = 1 / np.arange(1, optim_days+1)[::-1]  # Recent data is more heavily weighted
    msle_cases = np.sqrt(mean_squared_log_error(y_true_cases[-optim_days:], y_pred_cases[-optim_days:], weights))
    msle_fat = np.sqrt(mean_squared_log_error(y_true_fat[-optim_days:], y_pred_fat[-optim_days:], weights))
    
    msle_final = np.mean([msle_cases, msle_fat])
    
    if return_solution:
        return msle_final, sol
    else:
        return msle_final

The function below is essentially the same as above, by R is decayed using a Hill decay function. This model requires 2 additional parameters to be optimized, `k` & `L`

In [ ]:
# Use a Hill decayed reproduction number
def eval_model_decay(params, data, population, return_solution=False, forecast_days=0, optim_days=OPTIM_DAYS):
    R_0, t_hosp, t_crit, m, c, f, k, L = params  
    N = population
    n_infected = data['ConfirmedCases'].iloc[0]
    max_days = len(data) + forecast_days
    
    # https://github.com/SwissTPH/openmalaria/wiki/ModelDecayFunctions   
    # Hill decay. Initial values: R_0=2.2, k=2, L=50
    def time_varying_reproduction(t): 
        return R_0 / (1 + (t/L)**k)
    
    initial_state = [(N - n_infected)/ N, 0, n_infected / N, 0, 0, 0, 0]
    args = (time_varying_reproduction, 5.2, 2.9, t_hosp, t_crit, m, c, f)
            
    sol = solve_ivp(SEIR_HCD_model, [0, max_days], initial_state, args=args, t_eval=np.arange(0, max_days))
    
    sus, exp, inf, rec, hosp, crit, deaths = sol.y
    
    y_pred_cases = np.clip(inf + rec + hosp + crit + deaths, 0, np.inf) * population
    y_true_cases = data['ConfirmedCases'].values
    y_pred_fat = np.clip(deaths, 0, np.inf) * population
    y_true_fat = data['Fatalities'].values
    
    optim_days = min(optim_days, len(data))  # Days to optimise for
    weights = 1 / np.arange(1, optim_days+1)[::-1]  # Recent data is more heavily weighted
    
    msle_cases = np.sqrt(mean_squared_log_error(y_true_cases[-optim_days:], y_pred_cases[-optim_days:], weights))
    msle_fat = np.sqrt(mean_squared_log_error(y_true_fat[-optim_days:], y_pred_fat[-optim_days:], weights))
    msle_final = np.mean([msle_cases, msle_fat])
    
    if return_solution:
        return msle_final, sol
    else:
        return msle_final

In [ ]:
def use_last_value(train_data, valid_data, test_data):
    lv = train_data[['ConfirmedCases', 'Fatalities']].iloc[-1].values
    
    forecast_ids = test_data['ForecastId']
    submission.loc[forecast_ids, ['ConfirmedCases', 'Fatalities']] = lv
    
    if valid_data is not None:
        y_pred_valid = np.ones((len(valid_data), 2)) * lv.reshape(1, 2)
        y_true_valid = valid_data[['ConfirmedCases', 'Fatalities']]

        msle_cases = np.sqrt(mean_squared_log_error(y_true_valid['ConfirmedCases'], y_pred_valid[:, 0]))
        msle_fat = np.sqrt(mean_squared_log_error(y_true_valid['Fatalities'], y_pred_valid[:, 1]))
        msle_final = np.mean([msle_cases, msle_fat])

        return msle_final

In [ ]:
def plot_model_results(y_pred, train_data, valid_data=None, area_name=None):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16,5))

    if area_name is None:
        area_name = 'unknown'

    try:
        population = province_lookup[area_name]
    except KeyError:
        population = country_lookup[area_name]
    
    population /= 1E+06
    ax1.set_title(f'{area_name}, pop. = {population:.4f} M: Confirmed Cases')
    ax2.set_title(f'{area_name}, pop. = {population:.4f} M: Fatalities')
    
    train_data['ConfirmedCases'].plot(label='Confirmed Cases (train)', color='g', ax=ax1)
    y_pred.loc[train_data.index, 'ConfirmedCases'].plot(label='Modeled Cases', color='r', ax=ax1)
    ax3 = y_pred['R'].plot(label='Reproduction number', color='c', linestyle='-', secondary_y=True, ax=ax1)
    ax3.set_ylabel("Reproduction number", fontsize=10, color='c');
        
    train_data['Fatalities'].plot(label='Fatalities (train)', color='g', ax=ax2)
    y_pred.loc[train_data.index, 'Fatalities'].plot(label='Modeled Fatalities', color='r', ax=ax2)
    
    if valid_data is not None:
        valid_data['ConfirmedCases'].plot(label='Confirmed Cases (valid)', color='g', linestyle=':', ax=ax1)
        valid_data['Fatalities'].plot(label='Fatalities (valid)', color='g', linestyle=':', ax=ax2)
        y_pred.loc[valid_data.index, 'ConfirmedCases'].plot(label='Modeled Cases (forecast)', color='r', linestyle=':', ax=ax1)
        y_pred.loc[valid_data.index, 'Fatalities'].plot(label='Modeled Fatalities (forecast)', color='r', linestyle=':', ax=ax2)
    else:
        y_pred.loc[:, 'ConfirmedCases'].plot(label='Modeled Cases (forecast)', color='r', linestyle=':', ax=ax1)
        y_pred.loc[:, 'Fatalities'].plot(label='Modeled Fatalities (forecast)', color='r', linestyle=':', ax=ax2)
        
    ax1.legend(loc='best')
    

The function below fits a SEIR-HCD model for each area, either using a constant R or a decayed R, whichever is better. If the total cases/1M pop is below 1, then the last value is used.

In [ ]:
def fit_model_public(area_name, 
                     initial_guess=[2.2, 4, 14, 0.8, 0.1, 0.3, 2, 50],  # R_0, t_hosp, t_crit, m, c, f, k, l
                     bounds=PARAM_BOUNDS,
                     make_plot=True,
                     optim_days=OPTIM_DAYS):
        
    train_data = train.loc[area_name].query('ConfirmedCases > 0')
    valid_data = valid.loc[area_name]
    test_data = test_public.loc[area_name]  
    
    try:
        population = province_lookup[area_name]
    except KeyError:
        population = country_lookup[area_name]
        
    cases_per_million = train_data['ConfirmedCases'].max() * 10**6 / population
    n_infected = train_data['ConfirmedCases'].iloc[0]
        
    if cases_per_million < 1:
        return use_last_value(train_data, valid_data, test_data)
                
    res_const = minimize(eval_model_const, initial_guess[:-2], bounds=bounds[:-2],
                         args=(train_data, population, False, 0, optim_days),
                         method='L-BFGS-B')
    
    res_decay = minimize(eval_model_decay, initial_guess, bounds=bounds,
                         args=(train_data, population, False, 0, optim_days),
                         method='L-BFGS-B')
    
    dates_all = train_data.index.append(test_data.index)
    dates_val = train_data.index.append(valid_data.index)
    
    
    # If using a constant R number is better, use that model
    if res_const.fun < res_decay.fun:
        msle, sol = eval_model_const(res_const.x, train_data, population, True, len(test_data), optim_days)
        res = res_const
        R_t = pd.Series([res_const.x[0]] * len(dates_val), dates_val)
    else:
        msle, sol = eval_model_decay(res_decay.x, train_data, population, True, len(test_data), optim_days)
        res = res_decay
        
        # Calculate the R_t values
        t = np.arange(len(dates_val))
        R_0, t_hosp, t_crit, m, c, f, k, L = res.x  
        R_t = pd.Series(R_0 / (1 + (t/L)**k), dates_val)
        
    sus, exp, inf, rec, hosp, crit, deaths = sol.y
    
    y_pred = pd.DataFrame({
        'ConfirmedCases': np.clip(inf + rec + hosp + crit + deaths, 0, np.inf) * population,
        'Fatalities': np.clip(deaths, 0, np.inf) * population,
        'R': R_t,
    }, index=dates_all)
    
    y_pred_valid = y_pred.iloc[len(train_data): len(train_data)+len(valid_data)]
    y_pred_test = y_pred.iloc[len(train_data):]
    y_true_valid = valid_data[['ConfirmedCases', 'Fatalities']]
        
    valid_msle_cases = np.sqrt(mean_squared_log_error(y_true_valid['ConfirmedCases'], y_pred_valid['ConfirmedCases']))
    valid_msle_fat = np.sqrt(mean_squared_log_error(y_true_valid['Fatalities'], y_pred_valid['Fatalities']))
    valid_msle = np.mean([valid_msle_cases, valid_msle_fat])
    
    if make_plot:
        print(f'Validation RMSLE: {valid_msle:0.5f}')
        print(f'R: {res.x[0]:0.3f}, t_hosp: {res.x[1]:0.3f}, t_crit: {res.x[2]:0.3f}, '
              f'm: {res.x[3]:0.3f}, c: {res.x[4]:0.3f}, f: {res.x[5]:0.3f}')
        plot_model_results(y_pred, train_data, valid_data, area_name)
        
    # Put the forecast in the submission
    forecast_ids = test_data['ForecastId']
    submission.loc[forecast_ids, ['ConfirmedCases', 'Fatalities']] = y_pred_test[['ConfirmedCases', 'Fatalities']].values
    
    return valid_msle
            

In [ ]:
# Fit a model on the full dataset (i.e. no validation)
def fit_model_private(area_name, 
                      initial_guess=[2.2, 4, 14, 0.8, 0.1, 0.3, 2, 50],  # R_0, t_hosp, t_crit, m, c, f, k, l
                      bounds=PARAM_BOUNDS,
                      make_plot=True,
                      optim_days=OPTIM_DAYS):
        
    train_data = train_full.loc[area_name].query('ConfirmedCases > 0')
    test_data = test_private.loc[area_name]
    
    try:
        population = province_lookup[area_name]
    except KeyError:
        population = country_lookup[area_name]
        
    cases_per_million = train_data['ConfirmedCases'].max() * 10**6 / population
    n_infected = train_data['ConfirmedCases'].iloc[0]
        
    if cases_per_million < 1:
        return use_last_value(train_data, None, test_data)
                
    res_const = minimize(eval_model_const, initial_guess[:-2], bounds=bounds[:-2],
                         args=(train_data, population, False, 0, optim_days),
                         method='L-BFGS-B')
    
    res_decay = minimize(eval_model_decay, initial_guess, bounds=bounds,
                         args=(train_data, population, False, 0, optim_days),
                         method='L-BFGS-B')
    
    dates_all = train_data.index.append(test_data.index)
    
    
    # If using a constant R number is better, use that model
    if res_const.fun < res_decay.fun:
        msle, sol = eval_model_const(res_const.x, train_data, population, True, len(test_data), optim_days)
        res = res_const
        R_t = pd.Series([res_const.x[0]] * len(dates_all), dates_all)
    else:
        msle, sol = eval_model_decay(res_decay.x, train_data, population, True, len(test_data), optim_days)
        res = res_decay
        
        # Calculate the R_t values
        t = np.arange(len(dates_all))
        R_0, t_hosp, t_crit, m, c, f, k, L = res.x  
        R_t = pd.Series(R_0 / (1 + (t/L)**k), dates_all)
        
    sus, exp, inf, rec, hosp, crit, deaths = sol.y
    
    y_pred = pd.DataFrame({
        'ConfirmedCases': np.clip(inf + rec + hosp + crit + deaths, 0, np.inf) * population,
        'Fatalities': np.clip(deaths, 0, np.inf) * population,
        'R': R_t,
    }, index=dates_all)
    
    y_pred_test = y_pred.iloc[len(train_data):]
    
    if make_plot:
        print(f'R: {res.x[0]:0.3f}, t_hosp: {res.x[1]:0.3f}, t_crit: {res.x[2]:0.3f}, '
              f'm: {res.x[3]:0.3f}, c: {res.x[4]:0.3f}, f: {res.x[5]:0.3f}')
        plot_model_results(y_pred, train_data, area_name=area_name)
        
    # Put the forecast in the submission
    forecast_ids = test_data['ForecastId']
    submission.loc[forecast_ids, ['ConfirmedCases', 'Fatalities']] = y_pred_test[['ConfirmedCases', 'Fatalities']].values
            

In [ ]:
fit_model_public('Italy')
fit_model_private('Italy')

Above you can see the model optimized on the last 14 days of data from Italy which has been weighted to put more importance on recent data

The numbers show that `R_0` was around 3.5 and a decayed value is a better fit to the data. This is good news for Italy as it shows its measures are working.

Let's try Iran, South Korea and Japan. I chose these countries since they are probably the most challenging to model

In [ ]:
fit_model_public('Iran')
fit_model_private('Iran')

An unusual feature of the Iranian data is how high the R value needs to be for the model to fit. Iran was criticised for not closing religious shrines and locking down earlier, so the virus was extremely contagious. The data also appears of have multiple inflection points

In [ ]:
fit_model_public('Korea, South')
fit_model_private('Korea, South')

South Korea is unusual due to the incredible efforts to stop the spread

In [ ]:
fit_model_public('Japan')
fit_model_private('Japan')

I'm not sure why the red line is so wiggly for Japan ¯\\_(ツ)_/¯

In [ ]:
fit_model_public('Hubei')
fit_model_private('Hubei')

The plot for Hubei looks really good. In fact the data for most of the Chinese provinces fit well to compartment models

In [ ]:
fit_model_public('United Kingdom')
fit_model_private('United Kingdom')

Things are not looking good for the UK. Let's hope the lockdown starts bringing the R number down soon

In [ ]:
# large_msle = {
#     'Michigan': 10.155876, 
#     'Guadeloupe': 8.808455,
#     'Louisiana': 8.324547, 
#     'Uruguay': 7.921705,
#     'Panama': 7.201051, 
#     'Pakistan': 5.503595, 
#     'Equatorial Guinea': 5.103394,
#     'Massachusetts': 4.872891, 
#     'Mississippi': 4.70,
#     'Nova Scotia': 4.375212,
#     'Connecticut': 4.338,
#     'Slovenia': 4.321,
#     'Slovakia': 4.320,
#     'West Virginia': 4.284,
#     'Pennsylvania': 4.069,
#     'Peru': 3.930,
#     'Romania': 3.850,
#     'Argentina': 3.752,
#     'Ohio': 3.628,
#     'Wisconsin': 3.514
# }

# scores = []
# for cr, ref_score in large_msle.items():
#     print(cr, ref_score)
#     score = fit_model_public(cr, make_plot=False, bounds=PARAM_BOUNDS_SAFE)
#     print(score)
#     scores.append(score)
#     fit_model_private(cr, bounds=PARAM_BOUNDS_SAFE)
# print(np.mean(list(large_msle.values())))
# print(np.mean(scores))

In [ ]:
# AREA = 'Iran'

# def fit_predict_plot_logistic(area_name):
#     from scipy.optimize import curve_fit
#     from scipy.special import erf
#     trainval = pd.concat([train.reset_index(), valid.reset_index()], ignore_index=True)
#     try:
#         country_data = trainval.loc[trainval['Province_State'] == AREA, ['Date', 'ConfirmedCases', 'Fatalities']].set_index('Date').sort_index()
#         if not len(country_data) > 0:
#             raise ValueError
#     except:        
#         country_data = trainval.loc[trainval['Country_Region'] == AREA, ['Date', 'ConfirmedCases', 'Fatalities']].set_index('Date').sort_index()
#     country_data['n'] = range(len(country_data))

#     x_train = country_data.loc[country_data.index < '2020-03-19', 'n']
#     y_train = country_data.loc[country_data.index < '2020-03-19', 'ConfirmedCases']
#     x_val = country_data.loc[country_data.index >= '2020-03-19', 'n']
#     y_val = country_data.loc[country_data.index >= '2020-03-19', 'ConfirmedCases']

#     def log_curve(x, k, x_0, ymax):
#         return ymax / (1 + np.exp(-k*(x-x_0)))
    
#     def erf_func(t, p, a, beta, b):
#         return b + p/2 * erf(a * (t - beta))
    
#     try:
#         pop = country_lookup[area_name]
#     except KeyError:
#         pop = province_lookup[area_name]

#     # Fit the curve
#     # popt, pcov = curve_fit(log_curve, x_train, y_train, bounds=([0,0,0], [10, 120, np.inf]), maxfev=1000)
#     # estimated_k, estimated_x_0, ymax= popt
#     popt, pcov = curve_fit(erf_func, x_train, y_train, bounds=([0,0,0,0], [np.inf, np.inf, np.inf, np.inf]), maxfev=1000)
#     p, a, beta, b = popt    


#     # Plot the fitted curve
#     # Get predictions
# #     y_train_pred = [log_curve(i, k, x_0, ymax) for i in x_train]
# #     y_val_pred = [log_curve(i, k, x_0, ymax) for i in x_val]
# #     print(k, x_0, ymax)
#     y_train_pred = [erf_func(i, p, a, beta, b) for i in x_train]
#     y_train_pred = np.clip(y_train_pred, 0, pop)
#     y_val_pred = [erf_func(i, p, a, beta, b) for i in x_val]   
#     y_val_pred = np.clip(y_val_pred, 0, pop)
#     print(p, a, beta, b)

#     country_data['ConfirmedCases_pred'] = 0.
#     country_data.loc[country_data.index < '2020-03-19', 'ConfirmedCases_pred'] = y_train_pred
#     country_data.loc[country_data.index >= '2020-03-19', 'ConfirmedCases_pred'] = y_val_pred

#     score = mean_squared_log_error(country_data['ConfirmedCases'], country_data['ConfirmedCases_pred'])
#     print(score)

#     # # Plot everything for illustration
#     # fig = plt.figure()
#     # ax = fig.add_subplot(111)
#     # ax.plot(x_train, y_train_pred, 'r', '-', label='train pred')
#     # ax.plot(x_val, y_val_pred, 'r', '--', label='val pred')
#     # ax.plot(x_train, y_train, 'b', 'o', label='train truth')
#     # ax.plot(x_val, y_val, 'b', 'o', label='val truth')

#     # Refit on all the data
#     # Fit the curve
#     x, y = np.concatenate((x_train, x_val)), np.concatenate((y_train, y_val))
    
# #     popt, pcov = curve_fit(log_curve, x, y, bounds=([0,0,0],[10, 120, np.inf]), maxfev=1000)
# #     k, x_0, ymax= popt
    
#     popt, pcov = curve_fit(erf_func, x, y, bounds=([0,0,0,0], [np.inf, np.inf, np.inf, np.inf]), maxfev=1000)
#     p, a, beta, b = popt        

#     # Get predictions for test
#     te = test.loc[test['Date'] >= '2020-03-30', 'Date'].to_frame().copy()
#     # te['ConfirmedCases'] = 0.
#     te = te.set_index('Date').sort_index()
#     n_test = len(te)
#     start_test = np.max(x_val)
#     x_test = np.array(range(start_test, start_test + n_test))
#     # y_test_pred = [log_curve(i, k, x_0, ymax) for i in x_test]
#     y_test_pred = [erf_func(i, p, a, beta, b) for i in x_test]
#     y_test_pred = np.clip(y_test_pred, 0, pop)
#     x = np.concatenate((x, x_test))
#     y_pred = [erf_func(i, p, a, beta, b) for i in x]
#     y_pred = np.clip(y_pred, 0, pop)
#     # print(k, x_0, ymax)
#     print(p, a, beta, b)
#     te['ConfirmedCases_pred'] = y_test_pred

#     data_all = pd.concat([country_data.drop('n', axis=1), te], sort=True)
#     data_all['ConfirmedCases_pred'] = y_pred

#     country_data['ConfirmedCases'].plot(label='ConfirmedCases', color='blue')
#     country_data['ConfirmedCases_pred'].plot(label='valid pred', color='red', linestyle='--')
#     data_all['ConfirmedCases_pred'].plot(label='test pred', color='magenta')
#     plt.legend(loc='best')
    
# fit_predict_plot_logistic(AREA)

# Calculate for all countries

In [ ]:
# Initialize bounds and optim days to be used in fit
bound_dict = {}
optim_day_dict = {}

In [ ]:
# Public Leaderboard
validation_scores = []

for c in tqdm(test_public.index.levels[0].values):
    try:
        score = fit_model_public(c, make_plot=False)
        validation_scores.append({'Country': c, 'RMSLE': score})
        print(f'{c} {score:0.5f}')
    except IndexError as e:
        print(c, 'has no cases in train')
    except ValueError as e:
        print(c, e)
    bound_dict[c] = PARAM_BOUNDS
    optim_day_dict[c] = OPTIM_DAYS

validation_scores = pd.DataFrame(validation_scores)
print(f'Mean validation score: {np.sqrt(validation_scores["RMSLE"].mean()):0.5f}')

In [ ]:
validation_scores['RMSLE'].plot(kind='hist', bins=100)

In [ ]:
(validation_scores['RMSLE'] > 1.2).sum()

In [ ]:
# Public Leaderboard
validation_scores2 = []

for c in tqdm(test_public.index.levels[0].values):
    try:
        curr_score = validation_scores.loc[validation_scores['Country'] == c, 'RMSLE'].values[0]
    except IndexError as e:
        # print(' ', c, 'has no cases in train')
        continue
        
    if curr_score < 1.2:
        validation_scores2.append({'Country': c, 'RMSLE': curr_score})
        continue
    print(c)
    try:
        score = fit_model_public(c, make_plot=False, bounds=PARAM_BOUNDS_SAFE)
        validation_scores2.append({'Country': c, 'RMSLE': score})
        print(f' {score:0.5f} (was {curr_score:0.5f})')    
    except ValueError as e:
        print(c, e)
    bound_dict[c] = PARAM_BOUNDS_SAFE

validation_scores2 = pd.DataFrame(validation_scores2)
print(f'Mean validation score: {np.sqrt(validation_scores2["RMSLE"].mean()):0.5f}')

In [ ]:
validation_scores2['RMSLE'].plot(kind='hist', bins=100)

In [ ]:
(validation_scores2['RMSLE'] > 1.2).sum()

In [ ]:
# Public Leaderboard
validation_scores3 = []

for c in tqdm(test_public.index.levels[0].values):
    optim_day_dict[c] = 14
    try:
        curr_score = validation_scores2.loc[validation_scores['Country'] == c, 'RMSLE'].values[0]
    except IndexError as e:
        # print(' ', c, 'has no cases in train')
        continue
        
    if curr_score < 0.6:
        validation_scores3.append({'Country': c, 'RMSLE': curr_score})
        continue
    print(c)
    try:
        score = fit_model_public(c, make_plot=False, bounds=bound_dict[c], optim_days=10)
        if score < curr_score:
            optim_day_dict[c] = 10
            validation_scores3.append({'Country': c, 'RMSLE': score})
        else:
            validation_scores3.append({'Country': c, 'RMSLE': curr_score})
        print(f' {score:0.5f} (was {curr_score:0.5f})')    
    except ValueError as e:
        print(c, e)

validation_scores3 = pd.DataFrame(validation_scores3)
print(f'Mean validation score: {np.sqrt(validation_scores3["RMSLE"].mean()):0.5f}')

In [ ]:
# Public Leaderboard
validation_scores4 = []

for c in tqdm(test_public.index.levels[0].values):
    
    try:
        curr_score = validation_scores3.loc[validation_scores['Country'] == c, 'RMSLE'].values[0]
    except IndexError as e:
        # print(' ', c, 'has no cases in train')
        continue
        
    if curr_score < 0.6:
        validation_scores4.append({'Country': c, 'RMSLE': curr_score})
        continue
    print(c)
    try:        
        score = fit_model_public(c, make_plot=False, bounds=bound_dict[c], optim_days=21)
        if score < curr_score:
            optim_day_dict[c] = 21
            validation_scores4.append({'Country': c, 'RMSLE': score})
        else:
            validation_scores4.append({'Country': c, 'RMSLE': curr_score})
        print(f' {score:0.5f} (was {curr_score:0.5f})')    
    except ValueError as e:
        print(c, e)

validation_scores4 = pd.DataFrame(validation_scores4)
print(f'Mean validation score: {np.sqrt(validation_scores4["RMSLE"].mean()):0.5f}')

In [ ]:
# Find which areas are not being predicted well
validation_scores4.sort_values(by=['RMSLE'], ascending=False).head(20)

In [ ]:
# Some hardcoded info
add_day_dict = {
    c: 0 for c in test_public.index.levels[0].values
}

add_day_dict['Mississippi'] = 14
add_day_dict['Peru'] = 14
add_day_dict['Pennsylvania'] = 14
add_day_dict['West Virginia'] = 7
add_day_dict['Ecuador'] = 7
add_day_dict['Serbia'] = 7
add_day_dict['India'] = 7

In [ ]:
# Private Leaderboard
for c in tqdm(test_private.index.levels[0].values):
    try:
        score = fit_model_private(c, make_plot=False, bounds=bound_dict[c], optim_days=optim_day_dict[c] + add_day_dict[c])
    except IndexError as e:
        print(c, 'has no cases in train')

In [ ]:
submission.round().to_csv('submission.csv')

In [ ]:
submission.join(test.set_index('ForecastId')).query(f'Date > "{DATE_BORDER}"').round().to_csv('forecast.csv')

# Todo/ideas
* Some of the search boundaries boundaries are dependent on each other (e.g. `t_hosp` should always be less than `t_crit`). Using `NonlinearConstraint` may solve this
* Time dependence of other parameters, e.g. `f` as hospitals get overwhelmed
* Mix in other sources of data (e.g. number of hospital beds, age demographics)
* Global optmisation of virus specific parameters
* Use as features into a different model

# Thoughts on compartment models
I think compartment models are very powerful toy models and add lots of value for scenario testing, e.g. when a healthcare system might be overwhelmed, the impact of various interventions etc.. They can also be easily bootstrapped using population demographics to generate uncertainty estimates on those scenarios. 

The hospitalization compartment in this model is tricky to work with as I had to set the lower bounds for these transition times quite low (`t_hosp`, `t_crit`). I think the reason for this is that many countries are telling people to stay away from the hospitals unless they are experiencing severe symptoms, meaning that once a patient is in a hospital, there are likely to already be very sick. This means when I optimize the transition times to go from hospitalized > severe, the times are often very small.

However, I think that for this particular forecasting exercise and metric, there are more accurate methods that can be used. One could also use a compartment model like this to generate features for another model.
